In [ ]:
# Import all the libraries needed
import pandas as pd
from scipy import stats
import numpy as np
from numpy import mean
from numpy import std
from functools import reduce
from statistics import median

# Run previous ipynb files First then call MAIN_GRID
%run extractData.ipynb

In [ ]:
# Define constants
YEAR_STAR = 1959
YEAR_END  = 2015

# Calculate average of "57 years"
def getAvg(df, col):
    return df[col] - np.mean(df[col])

# Filter dataset
def filterDf(df, col):
    df = df.iloc[0:,[0,1]]
    return (df.set_index([col]).loc[YEAR_STAR:YEAR_END]).reset_index()

# Convert Year to datatime format
def convertYear(df, col):
    return pd.to_datetime(df[col], format="%Y")

# Hypothesis Test for normality
def isNormal(df, col):
    k2, p = stats.normaltest(df[col])
    alpha = 1e-3
    print("p = {:g}".format(p))
    # null hypothesis: x comes from a normal distribution
    if p < alpha:  
        print("The null hypothesis can be rejected for " + str(col))
    else:
        print("The null hypothesis cannot be rejected for " + str(col))

def transformDf(df, col):
    df = df.rename(index=df[col])
    df = df.drop(columns=[col])
    year = []
    sector = []
    emission = []
    for i in df.columns:
        for j in df.index.values:
            year.append(i)
            sector.append(j)
            emission.append(df[i][j])
    return pd.DataFrame({'Year':year, 'Sector':sector, 'Emission':emission}) 

def getAverageVals(df):
    avgTotal = []
    for i in df.index.values:
        vals = []
        for j in df.columns:
            vals.append(df[j][i])
        avgTotal.append(sum(vals) / len(vals))
    return avgTotal

# Anomaly Detection https://help.ceda.ac.uk/article/4728-cru-data-python-example

In [ ]:
df_seaLevel = MAIN_GRID[2][1]

# Rename columns
df_seaLevel = df_seaLevel.rename(index=str, columns={
    'CSIRO - Adjusted sea level (inches)': "CSIRO_ASLinches", 
    'Year':'Year_Merge'}
)

# Filter Year
df_seaLevel_f = filterDf(df_seaLevel, "Year_Merge")

# towardsdatascience.com/handling-missing-values-with-pandas-b876bf6f008f
# ffill (forward fill) replaces missing values with the values in the previous row
# Handle missing values
df_seaLevel_f = df_seaLevel_f.fillna(axis=0, method = 'ffill', limit=2)

# Calculate average
df_seaLevel_f['Anomaly_CSIRO_ASLinches'] = getAvg(df_seaLevel_f, "CSIRO_ASLinches")

In [ ]:
df_precip = MAIN_GRID[0][0]

# Rename columns
df_precip = df_precip.rename(index=str, columns={
    "Rainfall - (MM)": "Rnf_MM", 
    " Year": "Year", })

# Filter Year
df_precip_f = filterDf(df_precip, "Year")

# Convert Year to proper format
df_precip_f['Year'] = convertYear(df_precip_f, "Year")

# Set Year as index
df_precip_f.set_index('Year',inplace=True)

# https://stackoverflow.com/questions/23859840/python-aggregate-by-month-and-calculate-average
# Alternative method to group
df_precip_f_avgGrp = df_precip_f.groupby(pd.Grouper(freq='Y')).mean()

# Calculate average
df_precip_f_avgGrp["Anomaly_RnfMM"] = getAvg(df_precip_f_avgGrp, "Rnf_MM")

# Remove index column
df_precip_f_avgGrp.reset_index(level=0, inplace=True)

# Extract Year to Year_Merge Column
df_precip_f_avgGrp['Year_Merge'] = pd.DatetimeIndex(df_precip_f_avgGrp['Year']).year

In [ ]:
df_temp = MAIN_GRID[0][1]

# Rename columns
df_temp = df_temp.rename(index=str, columns={
    "Temperature - (Celsius)":"Tmp_Cls",
    " Year": "Year"})

# Filter Year
df_temp_f = filterDf(df_temp, "Year")

# Convert Year to proper format
df_temp_f['Year'] = convertYear(df_temp_f, "Year")

# Set Year as index
df_temp_f.set_index('Year',inplace=True)

# Alternative method to group
df_temp_f_avgGrp = df_temp_f.groupby(pd.Grouper(freq='Y')).mean()

# Calculate average
df_temp_f_avgGrp['Anomaly_TmpCls'] = getAvg(df_temp_f_avgGrp, "Tmp_Cls")

# Remove index column
df_temp_f_avgGrp.reset_index(level=0, inplace=True)

# Extract Year to Year_Merge Column
df_temp_f_avgGrp['Year_Merge'] = pd.DatetimeIndex(df_temp_f_avgGrp['Year']).year

In [ ]:
df_ghg = MAIN_GRID[2][0]

# Rename columns
df_ghg = df_ghg.rename(index=str, columns={
    "Year (negative values = BC)": "Year", 
    "Mauna Loa, Hawaii": "MLHawai_CO2ppm"})

# Filter dataset
df_ghg_f = df_ghg.iloc[0:,[0,4]]

# https://stackoverflow.com/questions/47444999/check-if-column-contains-type-string-object
# Convert everything to numerical values 
df_ghg_f.loc[:, df_ghg_f.dtypes.eq('object')] = df_ghg_f.loc[:, df_ghg_f.dtypes.eq('object')].apply(pd.to_numeric, errors='coerce')

# Filter dataset
df_ghg_f = (df_ghg_f.set_index(['Year']).loc[1959:2015]).reset_index() 

# Convert Year to proper format
df_ghg_f['Year'] = convertYear(df_ghg_f, "Year")

# Calculate average
df_ghg_f['Anomaly_CO2ppm'] = getAvg(df_ghg_f, "MLHawai_CO2ppm")

# Extract Year to Year_Merge Column
df_ghg_f['Year_Merge'] = pd.DatetimeIndex(df_ghg_f['Year']).year

In [ ]:
# https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes

dfs_to_merge = [df_precip_f_avgGrp,
                df_temp_f_avgGrp,
                df_seaLevel_f,
                df_ghg_f]

# Merge dataframes 
df_merged = reduce(lambda left, right: 
    pd.merge(left, right, on=['Year_Merge'], how='outer'), dfs_to_merge
).fillna('void')

# Rearrange columns
df_merged_final = df_merged.iloc[0:,[3,1,2,5,6,7,8,10,11]]

# Summarize statistics
print('mean=%.3f median =%.3f stdv=%.3f' % (mean(df_precip_f["Rnf_MM"]), median(df_precip_f["Rnf_MM"]), std(df_precip_f["Rnf_MM"])))

In [ ]:
isNormal(df_precip_f, "Rnf_MM")
isNormal(df_ghg_f, "MLHawai_CO2ppm")

In [ ]:
emisAll = MAIN_GRID[1][2]
emisAll = emisAll.drop(emisAll[emisAll["Economic Sector"] == "Total"].index)
emisAll = transformDf(emisAll, "Economic Sector")

emisTotal = MAIN_GRID[1][2]
emisTotal = transformDf(emisTotal, "Economic Sector")

for idx, row in emisTotal.iterrows():
    if row["Sector"] != "Total":
        emisTotal = emisTotal.drop([idx])

In [ ]:
emisBySector = MAIN_GRID[1][2]
colToDrop = "Economic Sector"
emisBySector = emisBySector.rename(index=emisBySector[colToDrop])
emisBySector = emisBySector.drop(emisBySector[emisBySector[colToDrop] == "Total"].index)
emisBySector = emisBySector.drop(columns=[colToDrop])

sectors = emisBySector.index.values
avgEmission = getAverageVals(emisBySector)

emisBySector = pd.DataFrame({"Emission": avgEmission}, index=sectors)

In [ ]:
emisByGas = MAIN_GRID[1][1]
colToDrop = "Gas"
emisByGas = emisByGas.rename(index=emisByGas[colToDrop])
emisByGas = emisByGas.drop(emisByGas[emisByGas[colToDrop] == "Total"].index)
emisByGas = emisByGas.drop(columns=[colToDrop])

sectors = emisByGas.index.values
avgEmission = getAverageVals(emisByGas)

emisByGas = pd.DataFrame({"Emission": avgEmission}, index=sectors)

In [ ]:
emisElectric = MAIN_GRID[1][4]
colToDrop = "Electricity Generation Sector"
emisElectric = emisElectric.rename(index=emisElectric[colToDrop])
emisElectric = emisElectric.drop(emisElectric[emisElectric[colToDrop] == "Total"].index)

emisElectric = transformDf(emisElectric, colToDrop)

In [ ]:
emisTransport = MAIN_GRID[1][7]
colToDrop = "Transportation Sector"
emisTransport = emisTransport.rename(index=emisTransport[colToDrop])
emisTransport = emisTransport.drop(emisTransport[emisTransport[colToDrop] == "Total"].index)

emisTransport = transformDf(emisTransport, colToDrop)